In [1]:
import pandas as pd
import numpy as np
import time
start = time.time()
##读取并合并数据
data_train = pd.read_csv("round1_ijcai_18_train_20180301.txt",sep=" ")
data_train = data_train.drop_duplicates()
data_test = pd.read_csv("round1_ijcai_18_test_a_20180301.txt",sep=" ")
data_all = pd.concat([data_train,data_test],axis=0)
data_train =[]
data_test = []
##排序后重新索引
data_all = data_all.sort_values('context_timestamp')
data_all = pd.DataFrame(data_all.values,columns=data_all.columns)
##不去除重复值和填补缺失值-1，将缺失值作为同类一起处理
#data_all = data_all.replace(to_replace = [-1],value=[np.nan])
length = len(data_all)
#将时间戳转换为时间信息,提取day和hour
import datetime
time_array = []
for i in range(length):
    time_list = []
    unix_ts = data_all.iloc[i]['context_timestamp']
    time = datetime.datetime.fromtimestamp(int(unix_ts))
    time_list.append(time.minute)
    time_list.append(time.day)
    time_list.append(time.hour)
    time_array.append(time_list)
time_array = pd.DataFrame(time_array,columns = ['min','day','hour'])
data_all.insert(0,'min',time_array['min'].values)
data_all.insert(0,'day',time_array['day'].values)
data_all.insert(0,'hour',time_array['hour'].values)
data_all.insert(0,'week_day',(time_array['day'].values)%7)

FileNotFoundError: File b'round1_ijcai_18_train_20180301.txt' does not exist

In [ ]:
col = ['item_brand_id','item_city_id','shop_id','user_id',
       'item_id','user_gender_id','user_occupation_id','item_category_list']
for co in col:
    se = data_all[co].drop_duplicates()
    se = pd.Series(se.index,index=se.values)
    data_all[co] = data_all[co].map(se)
data_all['time'] = ((data_all['day']-18)*24*60+(data_all['hour'])*60+data_all['min']).astype(int)
one = []
for i in range(len(data_all)):
    one.append(1)
data_all.insert(0,'cnt',one)
data_all.to_csv("data_for_click_statistics.csv",index=False)

In [ ]:
feature_click = pd.DataFrame()
data_all = pd.read_csv("data_for_click_statistics.csv")
feature_click.insert(0,'is_trade',data_all['is_trade'].values)
time = list(data_all['time'].drop_duplicates())
time_gap = [10,60,360,1440,2880]
col = ['item_brand_id','item_city_id','shop_id','user_id','hour',
       'item_id','user_gender_id','user_occupation_id','item_category_list']
for tg in time_gap:
    for co in col:
        data_all = pd.read_csv("data_for_click_statistics.csv")[[co,'time']]
        click = []
        for i in range(len(time)):
            data_before = data_all[(data_all['time']>time[i]-tg)&(data_all['time']<time[i])]
            data_now = data_all[data_all['time']==time[i]]
            if len(data_before)==0:
                for k in range(len(data_now)):
                    click.append(0)
            else:
                se = data_before[co].value_counts()
                click.extend(data_now[co].map(se).values)
            if i%100==0:
                print(i)
        col_name = co+"_click_last_"+str(tg)+"_minutes"
        print(tg,co,len(click))
        feature_click[col_name] = click
        feature_click.fillna(value=0,inplace=True)
        print(feature_click.corr()['is_trade'].sort_values())
        feature_click.to_csv("feature_click.csv",index=False)

In [ ]:
##点击次数组合

In [ ]:
import pandas as pd
import numpy as np
feature_click = pd.DataFrame()
data_all = pd.read_csv("data_for_click_statistics.csv")
feature_click.insert(0,'is_trade',data_all['is_trade'].values)
tim = list(data_all['time'].drop_duplicates())
time_gap = [10,60,360,1440,2880]
col = ['item_brand_id','item_city_id','shop_id','user_id','hour',
       'item_id','user_gender_id','user_occupation_id','item_category_list']

In [ ]:
n = len(col)
num = 0
for tg in time_gap:
    for a in range(n):
        for b in range(n-a-1):
            if num>116:
                import time
                start = time.time()
                col_new = [col[a],col[a+b+1]]
                col_new.append('time')
                col_new.append('cnt')
                data_all = pd.read_csv("data_for_click_statistics.csv")[col_new]
                click = []
                for i in range(len(tim)):
                    data_before = data_all[(data_all['time']>tim[i]-tg)&(data_all['time']<tim[i])]
                    data_now = data_all[data_all['time']==tim[i]]
                    if len(data_before)==0:
                        for k in range(len(data_now)):
                            click.append(0)
                    else:
                        se = data_before.groupby([col[a],col[a+b+1]])['cnt'].sum()
                        for r in range(len(data_now)):
                            ind = (data_now.iloc[r][col[a]],data_now.iloc[r][col[a+b+1]])
                            if ind in se.index:
                                click.append(se.loc[ind])
                            else:
                                click.append(0)
                    if i%100==0:
                        print(i)
                col_name = col[a]+"_"+col[a+b+1]+"_click_last_"+str(tg)+"_minutes"
                print(tg,col_new,len(click))
                feature_click[col_name] = click
                feature_click.fillna(value=0,inplace=True)
                print(feature_click.corr()['is_trade'].sort_values())
                feature_click.to_csv("feature_click_combine_2.csv",index=False)
                print(time.time()-start,"s")
            num+=1

In [ ]:
data_all.columns

In [ ]:
data_all = pd.read_csv("data_for_click_statistics.csv")